In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [4]:
from utils import generate_table, Cell

In [36]:
data = pd.read_csv("../../results/cmnist/2_digits/2colors/30seeds_base_post_iclr_submission.csv")

In [37]:
data["Group"].value_counts()

kmeans-fdm/cmnist/2digits/2colors/missing_s/baseline                 30
perfect-cluster.cmnist.2digits.2colors.mildly_subsampled.baseline    30
ranking-fdm.cmnist-2digits-2colors-missing_s-baseline                30
kmeans-fdm.cmnist-2digits-2colors-mildy_subsampled-oldencoder        30
/cmnist/2digits/2colors/missing_s/baseline                           30
kmeans-fdm.cmnist-2digits-2colors-missing_s-oldencoder               30
ranking-fdm.cmnist-2digits-2colors-mildly_sumbsampled-baseline       30
perfect-cluster/cmnist/2digits/2colors/missing_s/baseline            30
no-cluster-fdm.cmnist.2digits.2colors.mildly_subsampled.baseline     30
Name: Group, dtype: int64

In [38]:
data = data.replace({"Group": {
    "kmeans-fdm/cmnist/2digits/2colors/missing_s/baseline": "k-means (new encoder)",
    "perfect-cluster.cmnist.2digits.2colors.mildly_subsampled.baseline": "groundtruth cluster",
    "ranking-fdm.cmnist-2digits-2colors-missing_s-baseline": "ranking",
    "kmeans-fdm.cmnist-2digits-2colors-mildy_subsampled-oldencoder": "k-means (old encoder)",
    "/cmnist/2digits/2colors/missing_s/baseline": "no cluster",
    "kmeans-fdm.cmnist-2digits-2colors-missing_s-oldencoder": "k-means (old encoder)",
    "ranking-fdm.cmnist-2digits-2colors-mildly_sumbsampled-baseline": "ranking",
    "perfect-cluster/cmnist/2digits/2colors/missing_s/baseline": "groundtruth cluster",
    "no-cluster-fdm.cmnist.2digits.2colors.mildly_subsampled.baseline": "no cluster",
}}, inplace=False)

In [39]:
def merge(a, b):
    data[a] = data[a].combine_first(data[b])

merge("TPR_colour_0.0÷colour_1.0 (pytorch_classifier)", "TPR_colour_0.0/colour_1.0 (pytorch_classifier)")
merge("TNR_colour_0.0÷colour_1.0 (pytorch_classifier)", "TNR_colour_0.0/colour_1.0 (pytorch_classifier)")
merge('prob_pos_colour_0.0÷colour_1.0 (pytorch_classifier)', 'prob_pos_colour_0.0/colour_1.0 (pytorch_classifier)')

In [40]:
data.columns

Index(['Name', 'User', 'Created', 'Runtime', 'State', 'Notes', 'Tags', 'Group',
       'dataset', 'filter_labels', 'balanced_context', 'disc_weight',
       'elbo_weight', 'enc_levels', 'lr', 'num_disc_updates', 'num_discs',
       'seed', 'subsample_context.0', 'subsample_context.1',
       'subsample_context.2', 'subsample_context.3', 'subsample_train.0',
       'subsample_train.1', 'subsample_train.2', 'subsample_train.3',
       'zs_frac', 'enc_init_chans', 'enc_out_dim', 'kl_weight', 'oversample',
       'disc_method', 'Accuracy (pytorch_classifier)',
       'Clust/Context Accuracy', 'Clust/Context ARI',
       'Clust/Context Acc y=0 s=0', 'Clust/Context Acc y=0 s=1',
       'Clust/Context Acc y=1 s=0', 'Clust/Context Acc y=1 s=1',
       'Clust/Context NMI', 'Clust/Test ARI', 'Clust/Test Acc y=0 s=0',
       'Clust/Test Acc y=0 s=1', 'Clust/Test Acc y=1 s=0',
       'Clust/Test Acc y=1 s=1', 'Clust/Test Accuracy', 'Clust/Test NMI',
       'Renyi preds and s (pytorch_classifier)',

In [41]:
def table(filter_, cols):
    return (
        data
        .query(filter_)
        .groupby("Group")
        .agg(Cell(round_to=3))[list(cols)].rename(columns=cols, inplace=False)
    )

In [47]:
cols = {
    'Clust/Context Accuracy': "Clust Acc",
    'Clust/Context NMI': "Clust NMI",
    'Clust/Context ARI': "Clust ARI",
    "Accuracy (pytorch_classifier)": "Accuracy",
    'prob_pos_colour_0.0÷colour_1.0 (pytorch_classifier)': "AR ratio",
    "Renyi preds and s (pytorch_classifier)": "Renyi corr",
    "TPR_colour_0.0÷colour_1.0 (pytorch_classifier)": "TPR ratio",
    'TNR_colour_0.0÷colour_1.0 (pytorch_classifier)': "TNR ratio",
}

In [48]:
table("`subsample_train.0` == 1.0", cols)

,Clust Acc,Clust NMI,Clust ARI,Accuracy,AR ratio,Renyi corr,TPR ratio,TNR ratio
Group,,,,,,,,
groundtruth cluster,N/A,N/A,N/A,0.92 $\pm$ 0.058,0.686 $\pm$ 0.245,0.162 $\pm$ 0.138,0.684 $\pm$ 0.241,0.997 $\pm$ 0.003
k-means (old encoder),0.732 $\pm$ 0.157,0.723 $\pm$ 0.139,0.632 $\pm$ 0.2,0.904 $\pm$ 0.076,0.627 $\pm$ 0.308,0.196 $\pm$ 0.174,0.63 $\pm$ 0.313,0.995 $\pm$ 0.004
no cluster,N/A,N/A,N/A,0.829 $\pm$ 0.083,0.306 $\pm$ 0.337,0.381 $\pm$ 0.196,0.31 $\pm$ 0.342,0.994 $\pm$ 0.006
ranking,0.983 $\pm$ 0.006,0.939 $\pm$ 0.018,0.964 $\pm$ 0.011,0.912 $\pm$ 0.056,0.653 $\pm$ 0.232,0.178 $\pm$ 0.128,0.652 $\pm$ 0.227,0.995 $\pm$ 0.005


In [49]:
table("`subsample_train.0` == 0.0", cols)

,Clust Acc,Clust NMI,Clust ARI,Accuracy,AR ratio,Renyi corr,TPR ratio,TNR ratio
Group,,,,,,,,
groundtruth cluster,N/A,N/A,N/A,0.888 $\pm$ 0.088,0.854 $\pm$ 0.089,0.09 $\pm$ 0.064,0.852 $\pm$ 0.175,0.714 $\pm$ 0.203
k-means (new encoder),N/A,N/A,N/A,0.81 $\pm$ 0.132,0.751 $\pm$ 0.165,0.175 $\pm$ 0.138,0.712 $\pm$ 0.345,0.541 $\pm$ 0.308
k-means (old encoder),0.716 $\pm$ 0.153,0.723 $\pm$ 0.123,0.614 $\pm$ 0.193,0.796 $\pm$ 0.104,0.688 $\pm$ 0.219,0.227 $\pm$ 0.168,0.709 $\pm$ 0.328,0.492 $\pm$ 0.296
no cluster,N/A,N/A,N/A,0.851 $\pm$ 0.143,0.729 $\pm$ 0.257,0.148 $\pm$ 0.14,0.622 $\pm$ 0.405,0.797 $\pm$ 0.261
ranking,0.922 $\pm$ 0.024,0.836 $\pm$ 0.03,0.861 $\pm$ 0.032,0.868 $\pm$ 0.114,0.876 $\pm$ 0.097,0.076 $\pm$ 0.07,0.783 $\pm$ 0.265,0.702 $\pm$ 0.23
